In [6]:
import psycopg2
import os
from dotenv import load_dotenv

load_dotenv()

pg_host = os.getenv("PG_HOST")
pg_port = os.getenv("PG_PORT")
pg_db = os.getenv("PG_DB")
pg_user = os.getenv("PG_USER")
pg_password = os.getenv("PG_PASSWORD")

try:
    conn = psycopg2.connect(
        dbname=pg_db,
        user=pg_user,
        password=pg_password,
        host=pg_host,
        port=pg_port
    )
    print("Connection successful")
    conn.close()
except Exception as e:
    print(f"Connection failed: {e}")





Connection successful


In [7]:
from kafka import KafkaConsumer
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType
import json

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Kafka Stream Processing") \
    .getOrCreate()

# Kafka configuration
KAFKA_TOPIC = 'stock_data'
KAFKA_BOOTSTRAP_SERVERS = ['localhost:9092']

# Define Kafka consumer
consumer = KafkaConsumer(
    KAFKA_TOPIC,
    bootstrap_servers=KAFKA_BOOTSTRAP_SERVERS,
    group_id='stock_data_group',
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

# Define the schema for stock data
schema = StructType([
    StructField("open", FloatType(), True),
    StructField("high", FloatType(), True),
    StructField("low", FloatType(), True),
    StructField("close", FloatType(), True),
    StructField("volume", IntegerType(), True)
])

# PostgreSQL connection properties
pg_url = "jdbc:postgresql://localhost:5432/stockdb"
pg_properties = {
    "user": "postgres",
    "password": "Calion12.",
    "driver": "org.postgresql.Driver"
}

batch_size = 100  # Define batch size for processing
batch_data = []
messages_processed = 0

# Process each message
for message in consumer:
    try:
        message_value = message.value

        # Convert message to DataFrame
        new_data = [(float(message_value["1. open"]), float(message_value["2. high"]), float(message_value["3. low"]),
                     float(message_value["4. close"]), int(message_value["5. volume"]))]
        batch_data.extend(new_data)

        # Process in batches
        if len(batch_data) >= batch_size:
            new_df = spark.createDataFrame(batch_data, schema)
            
            # Show DataFrame for debugging
            new_df.show()
            
            # Write batch data to PostgreSQL
            new_df.write.jdbc(url=pg_url, table="stock_prices", mode="append", properties=pg_properties)
            batch_data = []  # Reset batch data
            messages_processed += batch_size

            # Log processing info
            print(f"Processed {messages_processed} messages")

    except Exception as e:
        print(f"Error processing message: {e}")

# Process any remaining data
if batch_data:
    new_df = spark.createDataFrame(batch_data, schema)
    new_df.write.jdbc(url=pg_url, table="stock_prices", mode="append", properties=pg_properties)

# Stop Spark session
spark.stop()


your 131072x1 screen size is bogus. expect trouble
24/08/03 13:24:01 WARN Utils: Your hostname, Zbook resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/03 13:24:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/03 13:24:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/03 13:24:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/03 13:24:19 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


KeyboardInterrupt: 

In [8]:
test_data = [(100.0, 110.0, 90.0, 105.0, 1000)]
test_df = spark.createDataFrame(test_data, schema)
test_df.write.jdbc(url=pg_url, table="stock_prices", mode="append", properties=pg_properties)


Py4JJavaError: An error occurred while calling o42.jdbc.
: java.lang.ClassNotFoundException: org.postgresql.Driver
	at java.base/java.net.URLClassLoader.findClass(URLClassLoader.java:445)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:592)
	at java.base/java.lang.ClassLoader.loadClass(ClassLoader.java:525)
	at org.apache.spark.sql.execution.datasources.jdbc.DriverRegistry$.register(DriverRegistry.scala:46)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.$anonfun$driverClass$1$adapted(JDBCOptions.scala:103)
	at scala.Option.foreach(Option.scala:407)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCOptions.<init>(JDBCOptions.scala:103)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:254)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcOptionsInWrite.<init>(JDBCOptions.scala:258)
	at org